In [10]:
import mysql.connector

# Connect to your MySQL database
conn = mysql.connector.connect(
    username='root',
    password='prabal9869',
    host='127.0.0.1',
    database='arl_bank'
)
cursor = conn.cursor()

# Example SQL query
query = """SELECT COUNT(*) AS Total_Withdrawals
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   AND Withdrawal_amount IS NOT NULL"""

# Execute the query
cursor.execute(query)

# Fetch all rows from the result set
rows = cursor.fetchall()

# Get column names from cursor description
column_names = [desc[0] for desc in cursor.description]

# Initialize an empty list to store dictionaries
results = []

# Iterate over the rows fetched from the database
for row in rows:
    # Create a dictionary for each row using column names
    result_dict = dict(zip(column_names, row))
    # Append the dictionary to the results list
    results.append(result_dict)

# Close cursor and connection
cursor.close()
conn.close()

# Print or further process the results list of dictionaries
print(results)


[{'Average_Daily_Spending': Decimal('18662884.0417')}]


In [23]:
from langchain import LLMChain, PromptTemplate
from langchain.tools.calculator import Calculator
from langchain_community.utilities import SQLDatabase
from langchain.agents import Tool, create_json_agent
from langchain_experimental.sql import SQLDatabaseChain

# Initialize LLM
llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q2_K.bin",
                    model_type="llama", config={'max_new_tokens': 3900, 'temperature': 0.2, 'context_length': 3800})

# Initialize Calculator Tool
calculator_tool = Calculator()

# Setup SQL Database
username = 'root'
password = 'prabal9869'
host = '127.0.0.1'
dbname = 'arl_bank'  # Database name
mysql_uri = f"mysql+pymysql://{username}:{password}@{host}/{dbname}"
db = SQLDatabase.from_uri(mysql_uri, sample_rows_in_table_info=3)

# Prompt for checking query completeness
query_evaluation_prompt = PromptTemplate(
    input_variables=["query"],
    template="""You are an expert financial data analyst.
Given the user's query: "{query}", determine if you have enough information to generate a SQL query to fetch the necessary data. If the query lacks specific information, such as an account number, date range, or transaction type, prompt the user to provide the missing details. If you have all the information needed, proceed to generate the SQL query.

Response:
"""
)

# Define the initial query evaluation chain
evaluation_chain = LLMChain(llm=llm, prompt=query_evaluation_prompt)

# SQL Execution Logic
def execute_sql_with_calculator(sql_query):
    # Use the SQL database chain to get the result
    sql_response = SQLDatabaseChain(db).run(sql_query)
    # Use calculator if needed (depending on the SQL response content)
    if 'calculate' in sql_response:
        return calculator_tool.run(sql_response)
    else:
        return sql_response

# Tools for agent
tools = [
    Tool(
        name="SQL Database",
        func=execute_sql_with_calculator,
        description="Executes SQL queries and performs calculations when needed."
    ),
    Tool(
        name="Calculator",
        func=calculator_tool.run,
        description="Performs mathematical calculations."
    ),
]

# Creating an agent that can use both tools
agent = create_json_agent(llm, tools)

# Function to handle user queries
def handle_user_query(user_question):
    # Evaluate if enough information is present
    evaluation_response = evaluation_chain.run(query=user_question)
    
    if "insufficient information" in evaluation_response:
        return evaluation_response  # Prompt user for more details
    else:
        # Proceed with SQL execution
        return agent(user_question)

user_question = "How much did I save this year?"
response = handle_user_query(user_question)
print(response)


ModuleNotFoundError: No module named 'langchain.tools.calculator'

In [3]:
from langchain.llms import CTransformers  # to load the llama2 model
llm = CTransformers(model="model\Openllama-3B-5Epoch-ProperTrainingdata.gguf",contex_length=3096,
                        model_type="llama", config={'temperature': 0.7, 'max_new_tokens': 800})

c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
from langchain.agents import load_tools
tools = load_tools(["llm-math"],llm=llm)


In [20]:
tools[0].name,tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [15]:
from langchain.agents import initialize_agent
agent = initialize_agent(tools,llm,agent="zero-shot-react-description",verbose=True)

c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [21]:
agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nCalculator(*args: Any, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

In [22]:
agent.run("Hey-how are you")

c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I am doing well, thank you
Action: I am going to buy a new phone
Action Input: my new phone
Observation: I am going to buy a new phone is not a valid tool, try one of [Calculator].
Thought: I should now invest this money on something else
Final Answer: I should now invest this money on something else is not a valid answer, try one of [Calculator].
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I am doing well, thank you
Final Answer: I am doing well, thank you
... (this Thought/Action/Action Input/Observation can repeat N times)

> Finished chain.


'I should now invest this money on something else is not a valid answer, try one of [Calculator].\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I am doing well, thank you\nFinal Answer: I am doing well, thank you\n... (this Thought/Action/Action Input/Observation can repeat N times)'

In [3]:
from langchain_community.utilities import SQLDatabase


# MySQL connection URI
username = 'root'
password = 'prabal9869'
host = '127.0.0.1'
dbname = 'arl_bank'  # Database name

# Constructing the MySQL URI
mysql_uri = f"mysql+pymysql://{username}:{password}@{host}/{dbname}"

# # Initializing SQLDatabase object for MySQL
db = SQLDatabase.from_uri(mysql_uri, sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE bank_processed5 (
	`Account_No` VARCHAR(25), 
	`Date` DATETIME, 
	`Transaction_details` TEXT, 
	`Value_date` DATETIME, 
	`Withdrawal_amount` INTEGER, 
	`Deposit_amount` INTEGER, 
	`Balance_amount` INTEGER
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from bank_processed5 table:
Account_No	Date	Transaction_details	Value_date	Withdrawal_amount	Deposit_amount	Balance_amount
409000611074'	2017-06-29 00:00:00	TRF FROM  Indiaforensic SERVICES	2017-06-29 00:00:00	0	1000000	1000000
409000611074'	2017-07-05 00:00:00	TRF FROM  Indiaforensic SERVICES	2017-07-05 00:00:00	0	1000000	2000000
409000611074'	2017-07-18 00:00:00	FDRL/INTERNAL FUND TRANSFE	2017-07-18 00:00:00	0	500000	2500000
*/


In [4]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm=llm,db=db,verbose=True)


# FEW SHORT LEARNING

In [5]:
few_shots=[
    {
        'Question':"What is my consistent income in my last 3 months.My Account Number is 409000493201'. ?",
        'SQLQuery':"""SELECT SUM(Deposit_amount) AS total_expenses FROM bank_processed5 WHERE Value_date BETWEEN DATE_SUB((SELECT MAX(Value_date) FROM bank_processed5), INTERVAL 3 MONTH) AND (SELECT MAX(Value_date) FROM bank_processed5) and Account_No="409000493201'";
        """,
        'SQLResult':"15802574",
        'Answer':"15802574"
    },
    {
        'Question':"What is the total balance for Account number 409000493201' ?",
        'SQLQuery':"""select sum(Balance_amount) from bank_processed5 where Account_No="409000611074'";

        """,
        'SQLResult':"1615533622",
        'Answer':"1615533622"
    }
]

In [6]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

to_vectorize = [" ".join(example.values()) for example in few_shots]

c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
to_vectorize 

['What is my consistent income in my last 3 months.My Account Number is 409000493201\'. ? SELECT SUM(Deposit_amount) AS total_expenses FROM bank_processed5 WHERE Value_date BETWEEN DATE_SUB((SELECT MAX(Value_date) FROM bank_processed5), INTERVAL 3 MONTH) AND (SELECT MAX(Value_date) FROM bank_processed5) and Account_No="409000493201\'";\n         15802574 15802574',
 'What is the total balance for Account number 409000493201\' ? select sum(Balance_amount) from bank_processed5 where Account_No="409000611074\'";\n\n         1615533622 1615533622']

In [8]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [9]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=1,
)
example_selector.select_examples({"Question": "What are my total expenses for the last 5 months?. My account number is 409000611074'?"})

[{'Answer': '15802574',
  'Question': "What is my consistent income in my last 3 months.My Account Number is 409000493201'. ?",
  'SQLQuery': 'SELECT SUM(Deposit_amount) AS total_expenses FROM bank_processed5 WHERE Value_date BETWEEN DATE_SUB((SELECT MAX(Value_date) FROM bank_processed5), INTERVAL 3 MONTH) AND (SELECT MAX(Value_date) FROM bank_processed5) and Account_No="409000493201\'";\n        ',
  'SQLResult': '15802574'}]

In [10]:
# Define the custom prompt for SQL database interactions
custom_mysql_prompt = """You are an expert in converting natural language into MySQL query.Only use the columns that are specified and needed to answer the question.
Use query for at most {top_k} results.
Table information:
1. `transactions` with columns: `Account_No`, `Date`, `Transaction_details`, `Value_date`, `Withdrawal_amount`, `Deposit_amount`, `Balance_amount`
"""

In [11]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX

In [12]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [13]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [14]:
suffix_eg="Question: {input}"

In [15]:
print(suffix_eg)

Question: {input}


In [16]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=custom_mysql_prompt,
    suffix=suffix_eg,
    input_variables=["input", "top_k"], #These variables are used in the prefix and suffix
)

In [17]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)


In [18]:
print(few_shot_prompt)

input_variables=['input', 'top_k'] example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001E2730A8990>, k=1, example_keys=None, input_keys=None, vectorstore_kwargs=None) example_prompt=PromptTemplate(input_variables=['Answer', 'Question', 'SQLQuery', 'SQLResult'], template='\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}') suffix='Question: {input}' prefix='You are an expert in converting natural language into MySQL query. Only use the columns that are needed to answer and specified in the question.\nUse query for at most {top_k} results.To calculate the last date first execute this query:"SET @last_date = \'2017-03-05\'" and to calculate the previous 3 month execute this query "SET @start_date = DATE_SUB(@last_date, INTERVAL 3 MONTH)" and only proceeded forward.\nTable information:\n1. `bank_processed5` with columns: `Account_No`, `Date`, `Transaction_details`, `Value_date`,

: 

In [19]:
result = new_chain("What are my total expenses for the last 5 months?. My account number is 409000611074'?")



> Entering new SQLDatabaseChain chain...
What are my total expenses for the last 5 months?. My account number is 409000611074'?
SQLQuery:

c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


SELECT SUM(Deposit_amount) AS total_expenses FROM bank_processed5 WHERE Value_date BETWEEN DATE_SUB((SELECT MAX(Value_date) FROM bank_processed5), INTERVAL 5 MONTH) AND (SELECT MAX(Value_date) FROM bank_processed5) and Account_No="409000611074'";
SQLResult: [(Decimal('41000000'),)]
Answer:

Number of tokens (513) exceeded maximum context length (512).
Number of tokens (514) exceeded maximum context length (512).
Number of tokens (515) exceeded maximum context length (512).
Number of tokens (516) exceeded maximum context length (512).
Number of tokens (517) exceeded maximum context length (512).
Number of tokens (518) exceeded maximum context length (512).
Number of tokens (519) exceeded maximum context length (512).
Number of tokens (520) exceeded maximum context length (512).
Number of tokens (521) exceeded maximum context length (512).
Number of tokens (522) exceeded maximum context length (512).
Number of tokens (523) exceeded maximum context length (512).
Number of tokens (524) exceeded maximum context length (512).
Number of tokens (525) exceeded maximum context length (512).
Number of tokens (526) exceeded maximum context length (512).
Number of tokens (527) exceeded maximum context length (512).
Number of tokens (528) exceeded maximum context length (512).
Number o

In [ ]:
print(result)

{'query': "What are my total expenses for the last three months?. My account number is 409000493201'?", 'result': 'None'}
